In [6]:
import collections
import os
corpus_file = os.path.join('Data', 'corpus.txt')

# Use a defaultdict to automatically handle new list creation
words_by_length = collections.defaultdict(list)

print(f"Starting to process {corpus_file}...")

# Check if the file exists before trying to open it
if not os.path.exists(corpus_file):
    print(f"Error: The file '{corpus_file}' was not found.")
    print("Please make sure your repository structure is correct.")
else:
    with open(corpus_file, 'r') as f:
        for line in f:
            # 1. Strip leading/trailing whitespace and convert to lowercase
            word = line.strip().lower()
            
            # 2. Filter out any lines that aren't purely alphabetic
            if word.isalpha():
                # 3. Group the word by its length
                words_by_length[len(word)].append(word)

    print("Corpus processing complete.")

    print(f"Found words for {len(words_by_length)} different lengths.")
    
    lengths_sorted = sorted(words_by_length.keys())
    
    if lengths_sorted:
        print("\n--- Word Counts Per Length (Sample) ---")
        sample_lengths = lengths_sorted[:5] + lengths_sorted[-5:]
        for length in sample_lengths:
            print(f"Length {length}: {len(words_by_length[length])} words")
            
        if 7 in words_by_length:
            print("\nExample (first 5 words of length 7):")
            print(words_by_length[7][:5])

Starting to process Data/corpus.txt...
Corpus processing complete.
Found words for 24 different lengths.

--- Word Counts Per Length (Sample) ---
Length 1: 46 words
Length 2: 84 words
Length 3: 388 words
Length 4: 1169 words
Length 5: 2340 words
Length 20: 40 words
Length 21: 16 words
Length 22: 8 words
Length 23: 3 words
Length 24: 1 words

Example (first 5 words of length 7):
['tunable', 'bajardo', 'emprise', 'frogbit', 'justina']


In [8]:
import string
import pickle
import collections
from collections import Counter

# --- Utility: Convert defaultdicts to normal dicts for pickling ---
def to_dict(obj):
    """Recursively convert defaultdicts to normal dicts for pickling."""
    if isinstance(obj, collections.defaultdict):
        obj = {k: to_dict(v) for k, v in obj.items()}
    elif isinstance(obj, dict):
        obj = {k: to_dict(v) for k, v in obj.items()}
    return obj

# --- HMM Configuration ---
models_output_file = 'hmm_models.pkl'  # Output file for trained models
ALPHABET = list(string.ascii_lowercase)
NUM_CLASSES = len(ALPHABET)

print("\nStarting HMM training...")

# Main model storage.
# Format: { length: {'pi': {...}, 'A': {...}} }
hmm_models = {}

for length, word_list in words_by_length.items():
    if not word_list or length < 2:
        continue 

    total_words = len(word_list)

    # --- Calculate Pi (Initial Probabilities) ---
    pi_counts = Counter(word[0] for word in word_list)
    pi_probs = {char: (pi_counts[char] + 1) / (total_words + NUM_CLASSES)
                for char in ALPHABET}

    # --- Calculate A (Transition Probabilities) ---
    A_counts = collections.defaultdict(lambda: collections.defaultdict(int))
    A_totals = collections.defaultdict(int)

    for word in word_list:
        for i in range(length - 1):
            prev_char = word[i]
            next_char = word[i + 1]
            A_counts[prev_char][next_char] += 1
            A_totals[prev_char] += 1

    A_probs = collections.defaultdict(lambda: collections.defaultdict(float))
    for prev_char in ALPHABET:
        total_prev_transitions = A_totals[prev_char]
        for next_char in ALPHABET:
            count = A_counts[prev_char][next_char]
            A_probs[prev_char][next_char] = (count + 1) / (total_prev_transitions + NUM_CLASSES)

    hmm_models[length] = {
        'pi': pi_probs,
        'A': A_probs
    }

print("\nHMM training complete.")
print(f"Trained {len(hmm_models)} models (one for each word length).")

# --- Convert all defaultdicts to dicts before saving ---
hmm_models_clean = to_dict(hmm_models)

# --- Save Models ---
with open(models_output_file, 'wb') as f:
    pickle.dump(hmm_models_clean, f)

print(f"Models saved to '{models_output_file}' successfully.")



Starting HMM training...

HMM training complete.
Trained 23 models (one for each word length).
Models saved to 'hmm_models.pkl' successfully.


In [9]:
# --- Phase 3: Save models to disk for later use ---
import collections

def to_dict(obj):
    """Recursively convert defaultdicts to normal dicts for pickling."""
    if isinstance(obj, collections.defaultdict):
        obj = {k: to_dict(v) for k, v in obj.items()}
    elif isinstance(obj, dict):
        obj = {k: to_dict(v) for k, v in obj.items()}
    return obj

# Convert defaultdicts before saving
hmm_models_clean = to_dict(hmm_models)

# Use pickle to serialize the main 'hmm_models' dict
try:
    with open(models_output_file, 'wb') as f:
        pickle.dump(hmm_models_clean, f)
    print(f"Successfully saved trained models to '{models_output_file}'")
except Exception as e:
    print(f"Error saving models: {e}")


Successfully saved trained models to 'hmm_models.pkl'


In [10]:
# --- Phase 4: Verification (Load models & check) ---
import pickle

try:
    with open(models_output_file, 'rb') as f:
        loaded_models = pickle.load(f)

    print(f"\n--- Verification: Loaded {len(loaded_models)} models ---")

    # Pick a common word length (e.g., 7)
    test_length = 7
    if test_length in loaded_models:
        model = loaded_models[test_length]
        print(f"\nChecking model for word length = {test_length}...")

        # --- Initial Probabilities ---
        print("\nInitial Probabilities (Pi) sample:")
        print(f"P('s'...) = {model['pi'].get('s', 0):.6f}")  # likely higher
        print(f"P('z'...) = {model['pi'].get('z', 0):.6f}")  # likely lower

        # --- Transition Probabilities ---
        print("\nTransition Probabilities (A) sample:")
        q_row = model['A'].get('q', {})
        if q_row:
            print(f"P('u' | 'q') = {q_row.get('u', 0):.6f}")
            print(f"P('x' | 'q') = {q_row.get('x', 0):.6f}")
        else:
            print("No transitions found for 'q' (possibly no words with 'q').")

        # --- Sanity check: normalization ---
        for ch in ['a', 'q', 's']:
            total_prob = sum(model['A'][ch].values())
            print(f"Sum of P(* | '{ch}') = {total_prob:.6f}")
            if abs(total_prob - 1.0) > 1e-3:
                print(f"Warning: probabilities for '{ch}' not normalized!")

    else:
        print(f"No model found for word length = {test_length}.")

except FileNotFoundError:
    print(f"Verification failed: '{models_output_file}' not found.")
except Exception as e:
    print(f"Verification failed: {e}")



--- Verification: Loaded 23 models ---

Checking model for word length = 7...

Initial Probabilities (Pi) sample:
P('s'...) = 0.109208
P('z'...) = 0.005840

Transition Probabilities (A) sample:
P('u' | 'q') = 0.614583
P('x' | 'q') = 0.010417
Sum of P(* | 'a') = 1.000000
Sum of P(* | 'q') = 1.000000
Sum of P(* | 's') = 1.000000
